<a href="https://colab.research.google.com/github/dorirozen/AI_Tasks/blob/main/NlpTask1_Dori_Shlomi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install beautifulsoup4
!pip install nltk

In [89]:
import nltk
import pandas as pd
import spacy
from bs4 import BeautifulSoup
from collections import Counter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [137]:
def readData():
  return pd.read_csv("/content/spam.csv", encoding="latin-1")
df = readData()

In [138]:
# Basic statistics
total_messages = len(df)
spam_messages = df["v1"].value_counts()["spam"]
ham_messages = df["v1"].value_counts()["ham"]



In [139]:
print(total_messages)
print(spam_messages)
print(ham_messages)

5572
747
4825


In [140]:
def preprocess_text(text):
    soup = BeautifulSoup(text, "html.parser")
    cleaned_text = soup.get_text()
    tokens = nltk.word_tokenize(cleaned_text.lower())
    return [token for token in tokens if token.isalpha()]

df["tokens"] = df["v2"].apply(preprocess_text)

<ipython-input-140-6aed0786977a>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [146]:
df["tokens"]
# tokenized_sms_spacy

0       [go, until, jurong, point, crazy, available, o...
1                          [ok, lar, joking, wif, u, oni]
2       [free, entry, in, a, wkly, comp, to, win, fa, ...
3       [u, dun, say, so, early, hor, u, c, already, t...
4       [nah, i, do, think, he, goes, to, usf, he, liv...
                              ...                        
5567    [this, is, the, time, we, have, tried, contact...
5568            [will, b, going, to, esplanade, fr, home]
5569    [pity, was, in, mood, for, that, so, any, othe...
5570    [the, guy, did, some, bitching, but, i, acted,...
5571                     [rofl, its, true, to, its, name]
Name: tokens, Length: 5572, dtype: object

In [96]:
# Average words per message
average_words = df["tokens"].apply(len).mean()
# Most frequent words
all_words = [word for tokens in df["tokens"] for word in tokens]
word_freq = Counter(all_words)
most_common_words = word_freq.most_common(5)

# Words that appear only once
unique_words = sum(1 for count in word_freq.values() if count == 1)

print(f"Total messages: {total_messages}")
print(f"Spam messages: {spam_messages}")
print(f"Ham messages: {ham_messages}")
print(f"Average words per message: {average_words:.2f}")
print(f"Most common words: {', '.join(word for word, _ in most_common_words)}")
print(f"Unique words appearing only once: {unique_words}")


Total messages: 5572
Spam messages: 747
Ham messages: 4825
Average words per message: 14.41
Most common words: i, to, you, a, the
Unique words appearing only once: 3594


In [148]:
#@title new for comparing with shlomi !
import csv
def csvfile(namefile):
    if ".csv" not in namefile:
        namefile += ".csv"
    csvfile = open(namefile, 'r', newline='', encoding = "ISO-8859-1")
    return list(csv.DictReader(csvfile))

spam = csvfile('/content/spam.csv')


from statistics import mean
def spam2text(spam):
  return [x['v2'] for x in spam]

def word_freq(text):
  word_dict = {}
  for string in text:
    for word in string.split(' '):
      try:
        word_dict[word] +=1
      except KeyError:
        word_dict[word] = 1

  return dict(sorted(word_dict.items(), key=lambda item: item[1], reverse=True))

def print_text_statistics(text):
  text_freq = word_freq(text)
  print(f"Word Count: {sum([len(x.split(' ')) for x in text])}\n" +
        f"Average Word Count: {mean([len(x.split(' ')) for x in text])}\n" +
        f"5 Most Frequent Words : {[freq for freq in text_freq.keys()][0:5]}\n" +
        f"Number of Rare Words : {len([freq for freq in text_freq.values() if freq==1])}\n")

print(f"Number of SMS Messages:{len(spam)}\n" +
      f"Number of Spams: {len([x for x in spam if x['v1']=='spam'])}")
spam_text = spam2text(spam)
print_text_statistics(spam_text)


Number of SMS Messages:5572
Number of Spams: 747
Word Count: 86961
Average Word Count: 15.60678391959799
5 Most Frequent Words : ['to', 'you', 'I', 'a', 'the']
Number of Rare Words : 9270



In [ ]:
#@title new version
from nltk.tokenize import word_tokenize
import time
# Tokenize the SMS text using NLTK
start_time = time.perf_counter()

tokenized_sms_nltk = [word_tokenize(sms) for sms in spam_text]
# tokenized_sms_nltk = [word_tokenize(sms) for sms in df["v2"]]

end_time = time.perf_counter()
execution_time = end_time - start_time
print(f"Nltk Execution time: {execution_time:.2f} seconds")


nlp = spacy.load("en_core_web_sm")
# Tokenize the SMS text using Spacy
start_time = time.perf_counter()

tokenized_sms_spacy = [[token.text for token in nlp(sms)] for sms in spam_text]
# tokenized_sms_spacy = [[token.text for token in nlp(sms)] for sms in df["v2"]]

end_time = time.perf_counter()
execution_time = end_time - start_time
print(f"Spacy Execution time: {execution_time:.2f} seconds")

print("NLTK Tokenization:")
sms_nltk = lambda i: print(tokenized_sms_nltk[i])
for i in range(4):
    sms_nltk(i)

print("spaCy Tokenization:")
sms_spacy = lambda i: print(tokenized_sms_spacy[i])
for i in range(4):
    sms_spacy(i)


In [149]:
from nltk.tokenize import word_tokenize
import time
# Tokenize the SMS text using NLTK
start_time = time.perf_counter()
tokenized_sms_nltk = [word_tokenize(sms) for sms in df["v2"]]
end_time = time.perf_counter()
execution_time = end_time - start_time
print(f"Nltk Execution time: {execution_time:.2f} seconds")


nlp = spacy.load("en_core_web_sm")
# Tokenize the SMS text using Spacy
start_time = time.perf_counter()
tokenized_sms_spacy = [[token.text for token in nlp(sms)] for sms in df["v2"]]
end_time = time.perf_counter()
execution_time = end_time - start_time
print(f"Spacy Execution time: {execution_time:.2f} seconds")

print("NLTK Tokenization:")
print(tokenized_sms_nltk)

print("spaCy Tokenization:")
print(tokenized_sms_spacy)

Nltk Execution time: 1.25 seconds
Spacy Execution time: 62.22 seconds
NLTK Tokenization:
['Go', 'until', 'jurong', 'point', ',', 'crazy', '..', 'Available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'Cine', 'there', 'got', 'amore', 'wat', '...']
['Ok', 'lar', '...', 'Joking', 'wif', 'u', 'oni', '...']
['Free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'FA', 'Cup', 'final', 'tkts', '21st', 'May', '2005', '.', 'Text', 'FA', 'to', '87121', 'to', 'receive', 'entry', 'question', '(', 'std', 'txt', 'rate', ')', 'T', '&', 'C', "'s", 'apply', '08452810075over18', "'s"]
['U', 'dun', 'say', 'so', 'early', 'hor', '...', 'U', 'c', 'already', 'then', 'say', '...']
['Nah', 'I', 'do', "n't", 'think', 'he', 'goes', 'to', 'usf', ',', 'he', 'lives', 'around', 'here', 'though']
spaCy Tokenization:
['Go', 'until', 'jurong', 'point', ',', 'crazy', '..', 'Available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'Cine', 'there', 'got', 

**NLTK Tokenization:** The word_tokenize function in NLTK uses the Punkt tokenizer which is based on unsupervised machine learning. The complexity is approximately 𝑂(𝑛), where 𝑛 is the number of characters in the text because it scans the text in a single pass.

**spaCy Tokenization:** spaCy's tokenizer is also 𝑂(𝑛) for similar reasons; it scans the text in a single pass. However, spaCy performs additional work like token attribute assignment and rule-based processing, which can add some constant time overhead, but does not change the linear complexity.

In [98]:
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

# Download the necessary NLTK resources
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to convert NLTK POS tags to WordNet POS tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


# Tokenize and lemmatize using NLTK
lemmatized_sms_nltk = []
for sms in df['v2']:
    tokens = word_tokenize(sms)
    pos_tags = pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(tag)) for token, tag in pos_tags]
    lemmatized_sms_nltk.append(lemmatized_tokens)

print("NLTK Lemmatization:")
print(lemmatized_sms_nltk)

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Lemmatize the SMS text using spaCy
lemmatized_sms_spacy = [[token.lemma_ for token in nlp(sms)] for sms in df['v2']]

print("spaCy Lemmatization:")
print(lemmatized_sms_spacy)

# Time Complexity Analysis:
# NLTK Lemmatization:
# - Tokenization: O(m), where m is the number of characters.
# - POS Tagging: O(n), where n is the number of tokens.
# - Lemmatization: O(n), where n is the number of tokens.
# Overall: O(m) + O(n) + O(n) = O(n), considering m ≈ n.

# spaCy Lemmatization:
# - Tokenization, POS Tagging, Lemmatization: O(n), where n is the number of tokens.
# Overall: O(n).

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


NLTK Lemmatization:
[['Go', 'until', 'jurong', 'point', ',', 'crazy', '..', 'Available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'Cine', 'there', 'get', 'amore', 'wat', '...'], ['Ok', 'lar', '...', 'Joking', 'wif', 'u', 'oni', '...'], ['Free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'FA', 'Cup', 'final', 'tkts', '21st', 'May', '2005', '.', 'Text', 'FA', 'to', '87121', 'to', 'receive', 'entry', 'question', '(', 'std', 'txt', 'rate', ')', 'T', '&', 'C', "'s", 'apply', '08452810075over18', "'s"], ['U', 'dun', 'say', 'so', 'early', 'hor', '...', 'U', 'c', 'already', 'then', 'say', '...'], ['Nah', 'I', 'do', "n't", 'think', 'he', 'go', 'to', 'usf', ',', 'he', 'live', 'around', 'here', 'though'], ['FreeMsg', 'Hey', 'there', 'darling', 'it', "'s", 'be', '3', 'week', "'s", 'now', 'and', 'no', 'word', 'back', '!', 'I', "'d", 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still', '?', 'Tb', 'ok', '!', 'XxX', 'std', 'chgs', 'to', 'send', ',', 'å£

**NLTK Lemmatization:** The time complexity of lemmatization in NLTK is
𝑂(𝑛), where 𝑛 is the number of tokens. This includes:

* **Tokenization:**
𝑂(𝑚), where 𝑚 is the number of characters.

* **POS Tagging:**
𝑂(𝑛), where 𝑛 is the number of tokens.

* **Lemmatization:**
𝑂(𝑛), where 𝑛 is the number of tokens.
The overall complexity is linear with respect to the number of characters and tokens.

**spaCy Lemmatization:** The time complexity of lemmatization in spaCy is also
𝑂(𝑛), where 𝑛 is the number of tokens. The spaCy pipeline includes:

* **Tokenization:**
𝑂(𝑚), where 𝑚 is the number of characters.

* **POS Tagging and Lemmatization:**
𝑂(𝑛), where 𝑛 is the number of tokens.
spaCy is highly optimized and processes text in a linear pass, making the overall complexity linear.

In [99]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer


# Download the necessary NLTK resources
nltk.download('punkt')

# Initialize the stemmer
stemmer = PorterStemmer()


# Tokenize and stem using NLTK
stemmed_sms_nltk = []
for sms in df["v2"]:
    tokens = word_tokenize(sms)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    stemmed_sms_nltk.append(stemmed_tokens)

print("NLTK Stemming:")
print(stemmed_sms_nltk)

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Tokenize using spaCy and stem using NLTK
stemmed_sms_spacy = []
for sms in df["v2"]:
    doc = nlp(sms)
    stemmed_tokens = [stemmer.stem(token.text) for token in doc]
    stemmed_sms_spacy.append(stemmed_tokens)

print("spaCy (with NLTK Stemming):")
print(stemmed_sms_spacy)

# Time Complexity Analysis:
# NLTK Stemming:
# - Tokenization: O(m), where m is the number of characters.
# - Stemming: O(n), where n is the number of tokens.
# Overall: O(m) + O(n) = O(n), considering m ≈ n.

# spaCy Stemming (using NLTK):
# - Tokenization: O(m), where m is the number of characters.
# - Stemming: O(n), where n is the number of tokens.
# Overall: O(n).

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


NLTK Stemming:
[['go', 'until', 'jurong', 'point', ',', 'crazi', '..', 'avail', 'onli', 'in', 'bugi', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'cine', 'there', 'got', 'amor', 'wat', '...'], ['ok', 'lar', '...', 'joke', 'wif', 'u', 'oni', '...'], ['free', 'entri', 'in', '2', 'a', 'wkli', 'comp', 'to', 'win', 'fa', 'cup', 'final', 'tkt', '21st', 'may', '2005', '.', 'text', 'fa', 'to', '87121', 'to', 'receiv', 'entri', 'question', '(', 'std', 'txt', 'rate', ')', 't', '&', 'c', "'s", 'appli', '08452810075over18', "'s"], ['u', 'dun', 'say', 'so', 'earli', 'hor', '...', 'u', 'c', 'alreadi', 'then', 'say', '...'], ['nah', 'i', 'do', "n't", 'think', 'he', 'goe', 'to', 'usf', ',', 'he', 'live', 'around', 'here', 'though'], ['freemsg', 'hey', 'there', 'darl', 'it', "'s", 'been', '3', 'week', "'s", 'now', 'and', 'no', 'word', 'back', '!', 'i', "'d", 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still', '?', 'tb', 'ok', '!', 'xxx', 'std', 'chg', 'to', 'send', ',', 'å£1.50', 'to', 'rc

**NLTK Stemming:** The time complexity of stemming in NLTK using the Porter stemmer is 𝑂(𝑛), where 𝑛
 is the number of tokens. The Porter stemmer processes each token independently in constant time relative to the length of the token, making the overall complexity linear with respect to the number of tokens.

**spaCy Stemming (using NLTK):** The time complexity for this combined approach involves two steps:
*  **Tokenization using spaCy:**
𝑂(𝑚), where 𝑚 is the number of characters.

*  **Stemming using NLTK:**
𝑂(𝑛), where 𝑛 is the number of tokens.
Since the stemming step dominates in terms of token count, the overall complexity remains 𝑂(𝑛).

**Tokenization:**

- NLTKs tokenization implementation is based on regular expressions and provides a simple list of tokens as output. spaCys tokenization implementation is more advanced, using a combination of rules and machine learning models, and provides a more structured output with additional information like part-of-speech tags and dependency relations.

- In terms of processing speed, spaCys tokenization is generally faster than NLTKs, especially for larger texts or corpora, due to its more efficient implementation and the use of machine learning models.

- Both NLTK and spaCy support tokenization for multiple languages, but spaCy has better support for a wider range of languages out-of-the-box, while NLTK requires additional language-specific resources to be installed.



**Lemmatization:**



- NLTKs lemmatization implementation uses the WordNet corpus and provides a simple list of lemmas as output. spaCys lemmatization implementation is integrated with its part-of-speech tagging and dependency parsing, providing a more structured output with additional information.

- spaCys lemmatization is generally faster than NLTKs, especially for larger texts or corpora, due to its more efficient implementation and the use of machine learning models.

- Both NLTK and spaCy support lemmatization for multiple languages, but spaCy has better support for a wider range of languages out-of-the-box, while NLTK requires additional language-specific resources to be installed.



**Stemming:**



- NLTKs stemming implementation uses the Porter Stemmer, a rule-based stemmer, and provides a simple list of stems as output. spaCys stemming implementation is integrated with its part-of-speech tagging and dependency parsing, providing a more structured output with additional information.

- In terms of processing speed, spaCys stemming is generally faster than NLTKs, especially for larger texts or corpora, due to its more efficient implementation and the use of machine learning models.

- Both NLTK and spaCy support stemming for multiple languages, but spaCy has better support for a wider range of languages out-of-the-box, while NLTK requires additional language-specific resources to be installed.



Overall, while NLTK provides a more straightforward and easy-to-use implementation for tokenization, lemmatization, and stemming, spaCy offers a more advanced and efficient implementation with better support for a wider range of languages and additional features like part-of-speech tagging and dependency parsing. However, the choice between NLTK and spaCy may depend on the specific requirements of the natural language processing task and the trade-off between simplicity and advanced features.







In [107]:
#@title print_text_statistics for list of lists
from collections import Counter

def analyze_array_data(arr):
    all_words = sum(len(sub_array) for sub_array in arr)
    word_frequencies = Counter(word for sub_array in arr for word in sub_array)
    average_count = sum(len(sub_array) for sub_array in arr) / len(arr) if arr else 0

    most_common_words = word_frequencies.most_common(5)
    unique_words = sum(1 for count in word_frequencies.values() if count == 1)

    print(f"Total words: {all_words}")
    # print(f"Word frequencies: {word_frequencies}")
    print(f"Average words per message: {average_count:.2f}")
    print(f"Most common words: {', '.join(word for word, _ in most_common_words)}")
    print(f"Unique words appearing only once: {unique_words}")

In [108]:
#@title print_text_statistics for dataFrame
import pandas as pd
from statistics import mean

def spam2text(df):
    return df['v2'].tolist()

def word_freq(text):
    word_dict = {}
    for string in text:
        for word in string.split():
            word_dict[word] = word_dict.get(word, 0) + 1
    return dict(sorted(word_dict.items(), key=lambda item: item[1], reverse=True))

def print_text_statistics(text):
    print("No TLS:\n")
    text_freq = word_freq(text)
    word_counts = [len(x.split()) for x in text]
    print(f"Word Count: {sum(word_counts)}\n" +
          f"Average Word Count: {mean(word_counts):.2f}\n" +
          f"5 Most Frequent Words: {[word for word in list(text_freq.keys())[:5]]}\n" +
          f"Number of Rare Words: {len([freq for freq in text_freq.values() if freq == 1])}\n")


In [109]:
def printC(df,ntlkText,spacyText):
  # print(f"Number of SMS Messages: {len(df)}\n" +f"Number of Spams: {len(df[df['v1'] == 'spam'])}")
  # print_text_statistics(spam2text(df))
  analyze_array_data(stemmed_sms_nltk)
  print("\n")
  analyze_array_data(stemmed_sms_spacy)



In [110]:
print("Stemming:\n\n")
printC(df,stemmed_sms_nltk,stemmed_sms_spacy)
print("\n\nLemmatization:\n\n")
printC(df,lemmatized_sms_nltk,lemmatized_sms_spacy)

Stemming:


Total words: 104193
Average words per message: 18.70
Most common words: ., i, to, you, ,
Unique words appearing only once: 4179


Total words: 103533
Average words per message: 18.58
Most common words: ., i, to, you, ,
Unique words appearing only once: 4252


Lemmatization:


Total words: 104193
Average words per message: 18.70
Most common words: ., i, to, you, ,
Unique words appearing only once: 4179


Total words: 103533
Average words per message: 18.58
Most common words: ., i, to, you, ,
Unique words appearing only once: 4252


In [ ]:
#@title code of shlomi from before
import pandas as pd

import nltk

from nltk.corpus import stopwords

import spacy

from bs4 import BeautifulSoup

from collections import Counter



# Load the spam.csv dataset

data = pd.read_csv(spam.csv, encoding=latin-1)


# Convert the text to lowercase and remove HTML tags

data[text] = data[v2].apply(lambda x: BeautifulSoup(x, html.parser).get_text().lower())



# Load the English stopwords

stop_words = set(stopwords.words(english))



# Initialize the spaCy English language model

nlp = spacy.load(en_core_web_sm)



# Tokenize the text and remove stopwords

tokenized_texts = []

for text in data[text]:

    doc = nlp(text)

    tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]

    tokenized_texts.append(tokens)



# Flatten the list of tokenized texts

all_tokens = [token for text in tokenized_texts for token in text]



# Print statistics after tokenization

print("After Tokenization:")

print(f"Total number of tokens: {len(all_tokens)}")

freq_dist = nltk.FreqDist(all_tokens)

print(f"5 most frequent words: {, .join(freq_dist.most_common(5))}")



# Lemmatize the tokens

lemmatized_tokens = []

for text in tokenized_texts:

    lemmas = [token.lemma_ for token in nlp( .join(text))]

    lemmatized_tokens.extend(lemmas)



# Print statistics after lemmatization

print("\nAfter Lemmatization:")

print(f"Total number of lemmas: {len(lemmatized_tokens)}")

freq_dist = nltk.FreqDist(lemmatized_tokens)

print(f"5 most frequent words: {, .join(freq_dist.most_common(5))}")



# Stem the tokens

stemmed_tokens = []

stemmer = nltk.stem.PorterStemmer()

for text in tokenized_texts:

    stems = [stemmer.stem(token) for token in text]

    stemmed_tokens.extend(stems)



# Print statistics after stemming

print("\nAfter Stemming:")

print(f"Total number of stems: {len(stemmed_tokens)}")

freq_dist = nltk.FreqDist(stemmed_tokens)

print(f"5 most frequent words: {, .join(freq_dist.most_common(5))}")


task 11

In [125]:
#@title printListsAfterActivation & token_stem_limmatizeing Functions
def printListsAfterActivation(text):
  tokenized_sentences = [word_tokenize(sentence) for sentence in text]

  lemmatized_sentences = []
  stemmed_sentences = []

  for sentence in tokenized_sentences:
    lemmatized_sentence = []
    stemmed_sentence = []


    doc = nlp(' '.join(sentence))
    lemmatized_sentence = [token.lemma_ for token in doc]


    stemmed_sentence = [stemmer.stem(token) for token in sentence]

    lemmatized_sentences.append(lemmatized_sentence)
    stemmed_sentences.append(stemmed_sentence)


  print("\nTokenized Sentences:")
  print(tokenized_sentences)

  print("\nLemmatized Sentences:")
  print(lemmatized_sentences)

  print("\nStemmed Sentences:")
  print(stemmed_sentences)

def token_stem_limmatizeing(text):
  tokenized_sentences = [word_tokenize(sentence) for sentence in text]
  lemmatized_sentences = []
  stemmed_sentences = []

  for sentence in tokenized_sentences:
    doc = nlp(' '.join(sentence))
    lemmatized_sentence = [token.lemma_ for token in doc]

    stemmed_sentence = [stemmer.stem(token) for token in sentence]

    lemmatized_sentences.append(lemmatized_sentence)
    stemmed_sentences.append(stemmed_sentence)

  print("Original Text Statistics:")
  analyze_array_data(tokenized_sentences)

  print("\nLemmatized Text Statistics:")
  analyze_array_data(lemmatized_sentences)

  print("\nStemmed Text Statistics:")
  analyze_array_data(stemmed_sentences)


In [126]:
import requests
from bs4 import BeautifulSoup

websites = ['https://afgprogrammer.com/flutter/',
            'https://huggingface.co/docs/transformers/model_doc/hubert']

scrap_sentences = []

for web in websites:
    soup = BeautifulSoup(requests.get(web).text,'html.parser')
    for p in soup.find_all('p'):
        scrap_sentences.append(p.text)


print("Scraped Sentences:")
print(scrap_sentences)

Scraped Sentences:
['Search in 100+ Flutter Examples ', 'Day 64 of Flutter 100 days of code.', 'Day 63 of Flutter 100 days of code.', 'Day 62 of Flutter 100 days of code.', 'Day 61 of Flutter 100 days of code.', 'Day 60 of Flutter 100 days of code.', 'Day 59 of Flutter 100 days of code.', 'Flutter Example.', 'Day 44 of Flutter 100 days of code.', 'Day 58 of Flutter 100 days of code.', 'Flutter Example.', 'Flutter Example.', 'Flutter Example.', 'Day 57 of Flutter 100 days of code.', 'Flutter Example', 'Day 38 of Flutter 100 days of code.', 'Day 57 of Flutter 100 days of code.', 'Day 46 of Flutter 100 days of code.', 'Day 35 of Flutter 100 days of code.', 'Flutter package example.', 'Flutter package example.', 'Day 35 of Flutter 100 days of code.', 'Flutter package example.', 'Flutter package example.', 'Day 35 of Flutter 100 days of code.', 'Flutter package example.', 'Flutter package example.', 'Day 56 of Flutter 100 days of code.', 'Day 45 of Flutter 100 days of code.', 'Flutter packa

In [129]:
printListsAfterActivation(scrap_sentences)


Tokenized Sentences:
[['Search', 'in', '100+', 'Flutter', 'Examples'], ['Day', '64', 'of', 'Flutter', '100', 'days', 'of', 'code', '.'], ['Day', '63', 'of', 'Flutter', '100', 'days', 'of', 'code', '.'], ['Day', '62', 'of', 'Flutter', '100', 'days', 'of', 'code', '.'], ['Day', '61', 'of', 'Flutter', '100', 'days', 'of', 'code', '.'], ['Day', '60', 'of', 'Flutter', '100', 'days', 'of', 'code', '.'], ['Day', '59', 'of', 'Flutter', '100', 'days', 'of', 'code', '.'], ['Flutter', 'Example', '.'], ['Day', '44', 'of', 'Flutter', '100', 'days', 'of', 'code', '.'], ['Day', '58', 'of', 'Flutter', '100', 'days', 'of', 'code', '.'], ['Flutter', 'Example', '.'], ['Flutter', 'Example', '.'], ['Flutter', 'Example', '.'], ['Day', '57', 'of', 'Flutter', '100', 'days', 'of', 'code', '.'], ['Flutter', 'Example'], ['Day', '38', 'of', 'Flutter', '100', 'days', 'of', 'code', '.'], ['Day', '57', 'of', 'Flutter', '100', 'days', 'of', 'code', '.'], ['Day', '46', 'of', 'Flutter', '100', 'days', 'of', 'code', '.

In [130]:
token_stem_limmatizeing(scrap_sentences)

Original Text Statistics:
Total words: 5736
Average words per message: 22.32
Most common words: ., ,, the, of, (
Unique words appearing only once: 296

Lemmatized Text Statistics:
Total words: 6290
Average words per message: 24.47
Most common words: ., the, ,, of, (
Unique words appearing only once: 257

Stemmed Text Statistics:
Total words: 5736
Average words per message: 22.32
Most common words: ., the, ,, of, (
Unique words appearing only once: 252


In [131]:
chat = open("/content/whatsup.txt", 'r')
chat_text = chat.readlines()
print(chat_text)


['הוא מוגש עם המסמך הסופי בתאריך שהוגדר לכם.\n', 'בדמו או בספר הפרויקט?\n', 'יש שם 2 שלבים\n', ' למיטב זכרוני, מועד הגשת הטיוטה הוא בשביל להגיע מוכנים לתערוכה, כלומר - עד אליו.\n', 'אחרי כן יהיה זמן לתיקונים קטנים מעטים במידת הצורך.\n', 'מישהו עשה תורת המשחקים?\n', 'מישהו יודע כמה אחוזים המטלות והפרויקט בשיטות ?\n', 'גם לכם לא מופיע עדיין ציון על פרוייקט גמר חלק ב במודל?\n', 'נכון\n', 'יש קבוצה לפיתוח אפליקציות מתקדם?\n', 'למשהוא חסר זוג לעבודה באלגוריתם מתקדם ?\n', ' החלון זמן לא אמור להסגר ביום שלישי? למה זה מראה את שבת?\n', 'עד סוף השבוע של השבוע השלישי לסמסטר.\n', ' מישהו שעושה C++ ואין לו עדיין זוג?\n', ' יש קבוצה לקורס הזה?\n', 'וואטסאפ הכוונה\n', '🤚\n', 'יודע אם הזוגות צריכים להיות באותו יום או שאפשר בימים שונים?\n', 'מי שבnlp מחר שרון ביטלה את השיעור \n', 'https://chat.whatsapp.com/IowiDyntvek2hhFJHakifl\n', 'גם אתם לקחתם חלק בקבלת הודעה ועדת המשמעת מאפקה ובפנים זה בכלל לגבי איזה מלגה של סטודנטית אקראית?\n', 'סעמקמקמק ירד לי הלב לתחתון איזה מערכת הודעות מפגרת\n', 'כןןןןן\n', 'כ

In [132]:
printListsAfterActivation(chat_text)


Tokenized Sentences:
[['הוא', 'מוגש', 'עם', 'המסמך', 'הסופי', 'בתאריך', 'שהוגדר', 'לכם', '.'], ['בדמו', 'או', 'בספר', 'הפרויקט', '?'], ['יש', 'שם', '2', 'שלבים'], ['למיטב', 'זכרוני', ',', 'מועד', 'הגשת', 'הטיוטה', 'הוא', 'בשביל', 'להגיע', 'מוכנים', 'לתערוכה', ',', 'כלומר', '-', 'עד', 'אליו', '.'], ['אחרי', 'כן', 'יהיה', 'זמן', 'לתיקונים', 'קטנים', 'מעטים', 'במידת', 'הצורך', '.'], ['מישהו', 'עשה', 'תורת', 'המשחקים', '?'], ['מישהו', 'יודע', 'כמה', 'אחוזים', 'המטלות', 'והפרויקט', 'בשיטות', '?'], ['גם', 'לכם', 'לא', 'מופיע', 'עדיין', 'ציון', 'על', 'פרוייקט', 'גמר', 'חלק', 'ב', 'במודל', '?'], ['נכון'], ['יש', 'קבוצה', 'לפיתוח', 'אפליקציות', 'מתקדם', '?'], ['למשהוא', 'חסר', 'זוג', 'לעבודה', 'באלגוריתם', 'מתקדם', '?'], ['החלון', 'זמן', 'לא', 'אמור', 'להסגר', 'ביום', 'שלישי', '?', 'למה', 'זה', 'מראה', 'את', 'שבת', '?'], ['עד', 'סוף', 'השבוע', 'של', 'השבוע', 'השלישי', 'לסמסטר', '.'], ['מישהו', 'שעושה', 'C++', 'ואין', 'לו', 'עדיין', 'זוג', '?'], ['יש', 'קבוצה', 'לקורס', 'הזה', '?'], ['וואטסאפ',

In [133]:
token_stem_limmatizeing(chat_text)

Original Text Statistics:
Total words: 1010
Average words per message: 7.54
Most common words: ?, של, את, יש, לא
Unique words appearing only once: 438

Lemmatized Text Statistics:
Total words: 1023
Average words per message: 7.63
Most common words: ?, של, את, יש, לא
Unique words appearing only once: 438

Stemmed Text Statistics:
Total words: 1010
Average words per message: 7.54
Most common words: ?, של, את, יש, לא
Unique words appearing only once: 436
